<a href="https://colab.research.google.com/github/SiriuXProtocoL/Tensorflow_examples/blob/main/01_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Basic classification: Classify images of clothing

In [ ]:
!pip install tensorflow

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


 Fashion MNIST dataset which contains 70,000 grayscale images in 10 categories. The images show individual articles of clothing at low resolution (28 by 28 pixels)

In [ ]:
#load the fashion MNIST dataset directly from tensorflow
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
print("Train Images Size", train_images.shape)
print("Train Images Label Size", train_labels.shape)
print("Train Set : ", train_labels)

print("\n Test Images Size", test_images.shape)
print("Test Images Label Size", test_labels.shape)
print("Test Set : ", test_labels)



The images are 28x28 NumPy arrays, with pixel values ranging from 0 to 255. The labels are an array of integers, ranging from 0 to 9.

In [ ]:
#label calss columns are not included in the dataset so store them as a list
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [ ]:
plt.figure()
plt.imshow(train_images[0]) #plotting the 1st image
plt.colorbar()
plt.grid(False)
plt.show()

- pixel value range b/w 0 to 255 so training set and testing set should be processed the same way.
- Scale these values to a range of 0 to 1 before feeding them to the neural network model. To do so, divide the values by 255.

In [ ]:
train_images = train_images / 255.0

test_images = test_images / 255.0


In [ ]:
#displaying the 1st 25 images with their corresponding class labels
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()


**Building the Model**
- tf.keras.layers.Dense, have parameters that are learned during training

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

- The first layer in this network, tf.keras.layers.Flatten, transforms the format of the images from a two-dimensional array (of 28 by 28 pixels) to a one-dimensional array (of 28 * 28 = 784 pixels).
-  tf.keras.layers.Dense layers. These are densely connected, or fully connected, neural layers. 
- The first Dense layer has 128 nodes (or neurons). The second (and last) layer returns a logits array with length of 10.
-  Each node contains a score that indicates the current image belongs to one of the 10 classes.

**Compile the model**

- Loss function —This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.
- Optimizer —This is how the model is updated based on the data it sees and its loss function.
- Metrics —Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified.


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

**Train the model**
- the training data is in the train_images and train_labels arrays.

In [ ]:
model.fit(train_images, train_labels, epochs=10)


**Evaluate accuracy**
- compare how the model performs on the test dataset

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)


- This gap between training accuracy and test accuracy represents overfitting

**Make predictions**

- The model is linear outputs, logits.
- Attach a softmax layer to convert the logits to probabilities, which are easier to interpret

In [ ]:
#Applying Softmax
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
#Assigning predictions
predictions = probability_model.predict(test_images)

In [ ]:
#making prediction for the 1st example
predictions[0]

- The prediction return the confidence level of all the categories

In [ ]:
#maximum probability amoung the prediction
np.argmax(predictions[0])


In [ ]:
#Original class label for the prediction
test_labels[0]

- Our model predicted correctly as ankle boot
- Creating a Graph of the confidence matrix for easy visualizations

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')


- Verify predictions
- correct predictions are blue and incorrect ones are red

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()


In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()


In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()


**Use the trained model**

- use the trained model to make a prediction about a single image

In [ ]:
# Grab an image from the test dataset.
img = test_images[1]

print(img.shape)


In [ ]:
# Add the image to a batch where it's the only member.
img = (np.expand_dims(img,0))

print(img.shape)


In [ ]:
#predict the labels
predictions_single = probability_model.predict(img)

print(predictions_single)


In [ ]:
#plot
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)


- tf.keras.Model.predict returns a list of lists—one list for each image in the batch of data. Grab the predictions for our (only) image in the batch:

In [ ]:
np.argmax(predictions_single[0])


Superb We got our predictions Right